In [1]:
import pandas as pd
import numpy as np
import sys, os

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
options = pd.read_csv("data/options.csv")

In [3]:
options.head()

,date_current,ticker,expiration_date,days_to_expiry,option_id,option_type,strike_price,bid_price,option_price,ask_price,implied_volatility,volume,open_interest,stock_price,dividend_yield,tdays_to_expiry,rate,bivs,nivs
0,2019-11-06,SPY,2019-11-08,2,SPY 2019-11-08 C235,C,235.0,72.31,72.51,72.44,180.66,0,1,307.07,1.81,2,0.0292,1.533355,0.00000
1,2019-11-06,SPY,2019-11-08,2,SPY 2019-11-08 C245,C,245.0,62.30,45.79,62.43,155.37,0,0,307.07,1.81,2,0.0292,1.319855,0.00000
2,2019-11-06,SPY,2019-11-08,2,SPY 2019-11-08 C250,C,250.0,57.30,49.31,57.44,143.75,1,1,307.07,1.81,2,0.0292,1.221679,0.00000
3,2019-11-06,SPY,2019-11-08,2,SPY 2019-11-08 C260,C,260.0,47.31,47.62,47.44,120.51,0,10,307.07,1.81,2,0.0292,1.024587,0.00000
4,2019-11-06,SPY,2019-11-08,2,SPY 2019-11-08 C265,C,265.0,42.31,33.93,42.44,108.79,0,0,307.07,1.81,2,0.0292,0.925710,0.92571


### Filter Regular Expirations

In [178]:
time_anchors = [
    1,
    3,
    6,
    9,
    12,
    18,
    24
]
time_anchors = [
    anchor * 21
    for anchor in time_anchors
]
time_anchors = np.array(time_anchors)

In [179]:
strike_anchors = list(range(80, 125, 5))
strike_anchors = np.array(strike_anchors)

### Dates

In [180]:
dates = options.date_current.unique()

### Try #1

In [109]:
for date in dates:
    
    tmp = options[options.date_current == date].reset_index(drop=True)
    tmp = tmp[tmp.tdays_to_expiry > 0]
    tmp = tmp[tmp.expiration_date.isin(regulars)]

    exps = tmp.tdays_to_expiry.unique()
    if exps.min() > 21:
        print(date, exps)
#     exps = exps.repeat(N_EXP).reshape(len(exps), N_EXP)
#     exps
    

2019-11-15 [ 24  42  65  85 148 211 231 256 275 293 336 399 462 526 549]
2019-11-18 [ 23  41  64  84 147 210 230 255 274 292 335 398 461 525 548]
2019-11-19 [ 22  40  63  83 146 209 229 254 273 291 334 397 460 524 547]
2020-01-17 [ 23  43  62  82 106 169 189 214 233 251 294 357 420 484 507 736]
2020-01-21 [ 22  42  81 105 168 188 213 232 250 293 356 419 483 506 735]
2020-05-15 [ 24  43  68  87 107 132 151 169 212 275 338 402 425 654]
2020-05-18 [ 23  42  67  86 106 131 150 168 211 274 337 401 424 463 653]
2020-05-19 [ 22  41  66  85 105 130 149 167 210 273 336 400 423 462 652]
2020-07-17 [ 25  44  64  89 108 126 169 232 295 359 382 421 484 611]
2020-07-20 [ 24  43  63  88 107 125 168 231 294 358 381 420 483 610]
2020-07-21 [ 23  42  62  87 106 124 167 230 293 357 380 419 482 609]
2020-07-22 [ 22  41  61  86 105 123 166 229 292 356 379 418 481 608]
2020-10-16 [ 25  44  62  85 105 124 149 168 231 295 318 357 420 483 547 569]
2020-10-19 [ 24  43  61  84 104 123 148 167 230 294 317 356 419

In [112]:
dates.tolist().index("2020-07-17")

172

In [117]:
tmp = options[options.date_current == dates[172]].reset_index(drop=True)
tmp = tmp[tmp.tdays_to_expiry > 0]

start = "2019-01-01"
end = "2029-01-01"

fridays = pd.date_range(start, end, freq="WOM-3FRI").astype(str)
thursdays = pd.date_range(start, end, freq="WOM-3THU").astype(str)
regulars = list(fridays) + list(thursdays)

tmp = tmp[tmp.expiration_date.isin(regulars)]

exps = tmp.tdays_to_expiry.unique()
exps = exps.repeat(N_EXP).reshape(len(exps), N_EXP)
exps

array([[ 25,  25,  25,  25,  25,  25,  25],
       [ 44,  44,  44,  44,  44,  44,  44],
       [ 64,  64,  64,  64,  64,  64,  64],
       [ 89,  89,  89,  89,  89,  89,  89],
       [108, 108, 108, 108, 108, 108, 108],
       [126, 126, 126, 126, 126, 126, 126],
       [169, 169, 169, 169, 169, 169, 169],
       [232, 232, 232, 232, 232, 232, 232],
       [295, 295, 295, 295, 295, 295, 295],
       [359, 359, 359, 359, 359, 359, 359],
       [382, 382, 382, 382, 382, 382, 382],
       [421, 421, 421, 421, 421, 421, 421],
       [484, 484, 484, 484, 484, 484, 484],
       [611, 611, 611, 611, 611, 611, 611]])

In [118]:
dist = exps - expirations
dist

array([[   4,  -38, -101, -164, -227, -353, -479],
       [  23,  -19,  -82, -145, -208, -334, -460],
       [  43,    1,  -62, -125, -188, -314, -440],
       [  68,   26,  -37, -100, -163, -289, -415],
       [  87,   45,  -18,  -81, -144, -270, -396],
       [ 105,   63,    0,  -63, -126, -252, -378],
       [ 148,  106,   43,  -20,  -83, -209, -335],
       [ 211,  169,  106,   43,  -20, -146, -272],
       [ 274,  232,  169,  106,   43,  -83, -209],
       [ 338,  296,  233,  170,  107,  -19, -145],
       [ 361,  319,  256,  193,  130,    4, -122],
       [ 400,  358,  295,  232,  169,   43,  -83],
       [ 463,  421,  358,  295,  232,  106,  -20],
       [ 590,  548,  485,  422,  359,  233,  107]])

In [119]:
sdist = np.sign(dist)
sdist

array([[ 1, -1, -1, -1, -1, -1, -1],
       [ 1, -1, -1, -1, -1, -1, -1],
       [ 1,  1, -1, -1, -1, -1, -1],
       [ 1,  1, -1, -1, -1, -1, -1],
       [ 1,  1, -1, -1, -1, -1, -1],
       [ 1,  1,  0, -1, -1, -1, -1],
       [ 1,  1,  1, -1, -1, -1, -1],
       [ 1,  1,  1,  1, -1, -1, -1],
       [ 1,  1,  1,  1,  1, -1, -1],
       [ 1,  1,  1,  1,  1, -1, -1],
       [ 1,  1,  1,  1,  1,  1, -1],
       [ 1,  1,  1,  1,  1,  1, -1],
       [ 1,  1,  1,  1,  1,  1, -1],
       [ 1,  1,  1,  1,  1,  1,  1]])

In [138]:
dsdist = np.diff(sdist, axis=0)

In [142]:
brackets = [
    (idc[1], idc[0], idc[0] + 1)
    for idc in np.argwhere(dsdist == 2)
]
brackets

[(1, 1, 2), (3, 6, 7), (4, 7, 8), (5, 9, 10), (6, 12, 13)]

In [143]:
brackets.extend([
    (idc[1], idc[0])
    for idc in np.argwhere(sdist == 0)
])

In [144]:
sorted(brackets)

[(1, 1, 2), (2, 5), (3, 6, 7), (4, 7, 8), (5, 9, 10), (6, 12, 13)]

### Brackets

In [239]:
def brackets(values, anchors):
    
    N = len(values)
    M = len(anchors)
    
    values = np.array(values)
    matrix = values.repeat(M).reshape(N, M)
    
    matrix -= anchors
    signed_matrix = np.sign(matrix)
    dsigned_matrix = np.diff(signed_matrix)
    
    return matrix, signed_matrix, dsigned_matrix

In [262]:
dates[-45:-30]

array(['2020-10-14', '2020-10-15', '2020-10-16', '2020-10-19',
       '2020-10-20', '2020-10-21', '2020-10-22', '2020-10-23',
       '2020-10-26', '2020-10-27', '2020-10-28', '2020-10-29',
       '2020-10-30', '2020-11-02', '2020-11-03'], dtype=object)

In [263]:
options[options.date_current.isin(dates[-45:-30])].reset_index(drop=True).to_csv("data/small_options.csv", index=False)

In [243]:
_options = options[options.date_current == dates[18]].reset_index(drop=True)
_options = _options[_options.tdays_to_expiry > 0]
_options = _options[_options.tdays_to_expiry == 2]

In [244]:
strikes = _options

,date_current,ticker,expiration_date,days_to_expiry,option_id,option_type,strike_price,bid_price,option_price,ask_price,implied_volatility,volume,open_interest,stock_price,dividend_yield,tdays_to_expiry,rate,bivs,nivs
0,2019-12-04,SPY,2019-12-06,2,SPY 2019-12-06 C265,C,265.0,46.29,45.71,46.42,0.00,2,0,311.5,1.81,2,0.030253,0.000000,0.000000
1,2019-12-04,SPY,2019-12-06,2,SPY 2019-12-06 C270,C,270.0,41.30,38.12,41.43,0.00,30,1,311.5,1.81,2,0.030253,0.000000,0.000000
2,2019-12-04,SPY,2019-12-06,2,SPY 2019-12-06 C275,C,275.0,36.30,36.63,36.43,0.00,80,35,311.5,1.81,2,0.030253,0.000000,0.000000
3,2019-12-04,SPY,2019-12-06,2,SPY 2019-12-06 C280,C,280.0,31.30,31.74,31.43,0.00,84,1,311.5,1.81,2,0.030253,0.000000,0.000000
4,2019-12-04,SPY,2019-12-06,2,SPY 2019-12-06 C281,C,281.0,30.32,31.10,30.43,0.00,147,12,311.5,1.81,2,0.030253,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2019-12-04,SPY,2019-12-06,2,SPY 2019-12-06 P329,P,329.0,17.92,17.41,17.74,45.61,0,0,311.5,1.81,2,0.030253,0.394035,0.394035
95,2019-12-04,SPY,2019-12-06,2,SPY 2019-12-06 P330,P,330.0,18.63,15.84,18.74,47.56,0,1,311.5,1.81,2,0.030253,0.357978,0.357978
96,2019-12-04,SPY,2019-12-06,2,SPY 2019-12-06 P332.5,P,332.5,21.12,21.04,21.24,52.44,0,10,311.5,1.81,2,0.030253,0.393299,0.393299
97,2019-12-04,SPY,2019-12-06,2,SPY 2019-12-06 P335,P,335.0,23.63,25.49,23.74,54.49,0,14,311.5,1.81,2,0.030253,0.432692,0.432692


### Time Brackets

In [237]:
def time_bracket(expirations, time_anchors):
    
    N = len(expirations)
    M = len(time_anchors)
    
    expirations_m = expirations.repeat(M).reshape(N, M)
    
    dist = expirations_m - time_anchors
    sdist = np.sign(dist)
    dsdist = np.diff(sdist, axis=0)
        
    brackets = [
        (
            idc[1],
            expirations[idc[0]],
            expirations[idc[0] + 1]
        )
        for idc in np.argwhere(dsdist == 2)
    ]
    brackets += [
        (
            idc[1],
            expirations[idc[0]]
        )
        for idc in np.argwhere(sdist == 0)
    ]
    
    return sorted(brackets), dsdist

_options = options[options.date_current == dates[18]].reset_index(drop=True)
_options = _options[_options.tdays_to_expiry > 0]

start = "2019-01-01"
end = "2029-01-01"

fridays = pd.date_range(start, end, freq="WOM-3FRI").astype(str)
thursdays = pd.date_range(start, end, freq="WOM-3THU").astype(str)
regulars = list(fridays) + list(thursdays)

expirations = _options[_options.expiration_date.isin(regulars)]
expirations = expirations.tdays_to_expiry.unique()

brackets, dsdist = time_bracket(expirations, time_anchors)

idc = np.argwhere(dsdist.sum(axis=0) == 0)
if len(idc) != 0:
    
    expirations = _options.tdays_to_expiry.unique()
    nr_brackets, _ = time_bracket(expirations, time_anchors[idc])
    brackets.extend(nr_brackets)

In [238]:
sorted(brackets)

[(0, 12, 30),
 (1, 53, 73),
 (2, 92, 136),
 (3, 136, 199),
 (4, 244, 263),
 (5, 324, 387),
 (6, 450, 514)]